# Alignment search
##### by Javier Lopez Segura

The following Python script will produce a graph showing the alignment of 4 genomes:
<br />    - Mycobacterium tuberculosis H37Ra
<br />    - Mycobacterium tuberculosis h37Rv
<br />    - Mycolicibacterium smegmatis MC2155
<br />    - Mycobacterium tuberculosis variant bovis AF2122/97

#### Part 1. Alignment functionality

In [127]:
from Bio.Seq import Seq
from Bio import pairwise2
from Bio.SeqRecord import SeqRecord
from Bio.Align import MultipleSeqAlignment
import pandas as pd
import os
from Bio.pairwise2 import format_alignment
from IPython.display import display, Markdown
from Bio.SeqUtils.ProtParam import ProteinAnalysis

import os, io, random
import string
import numpy as np
import re

from Bio.Seq import Seq
from Bio.Align import MultipleSeqAlignment
from Bio import AlignIO, SeqIO

import panel as pn
import panel.widgets as pnw
pn.extension()

from bokeh.plotting import figure
from bokeh.models import ColumnDataSource, Plot, Grid, Range1d
from bokeh.models.glyphs import Text, Rect
from bokeh.layouts import gridplot
from bokeh.io import show

# Opening the iformation from Mycobacterium tuberculosis variant bovis AF2122/97
path_to_bovis = "/Users/dissertation/Documents/Dissertation-real/cgi-mycoapollo/db/genomes/Bovis_data1.csv"
bovis_data = pd.read_csv(path_to_bovis, encoding='utf-8')
bovis_data1 = bovis_data.drop(['Unnamed: 0'], axis=1)
bovis_data1 = bovis_data1.replace(r'\n','', regex=True) 

# Opening the information from Mycobacterium tuberculosis H37Ra
path_to_H37RA = "/Users/dissertation/Documents/Dissertation-real/cgi-mycoapollo/db/genomes/H37RA_data1.csv"
H37RA_data = pd.read_csv(path_to_H37RA,encoding='utf-8')
H37RA_data1 = H37RA_data.drop(['Unnamed: 0'], axis=1)
H37RA_data1 = H37RA_data1.replace(r'\n','', regex=True) 

# Opening the information from Mycolicibacterium smegmatis MC2155
path_to_smagmatis = "/Users/dissertation/Documents/Dissertation-real/cgi-mycoapollo/db/genomes/smegmatis_data1.csv"
smagmatis_data = pd.read_csv(path_to_smagmatis,encoding='utf-8')
smagmatis_data1 = smagmatis_data.drop(['Unnamed: 0'], axis=1)
smagmatis_data1 = smagmatis_data1.replace(r'\n','', regex=True)

# Opening the information Mycobacterium tuberculosis H37Rv
path_to_tuberculosis = "/Users/dissertation/Documents/Dissertation-real/cgi-mycoapollo/db/genomes/Tuberculosis_data1.csv"
tuberculosis_data = pd.read_csv(path_to_tuberculosis,encoding='utf-8')
tuberculosis_data1 = tuberculosis_data.drop(['Unnamed: 0'], axis=1)
tuberculosis_data1 = tuberculosis_data1.replace(r'\n','', regex=True) 

# This file contains the information from the orthologs
path_orthologs = "/Users/dissertation/Documents/Dissertation-real/cgi-mycoapollo/db/genomes/locus_alignment_info.csv"
orthologs = pd.read_csv(path_orthologs)
orthologs = orthologs.applymap(lambda x: 'N/A' if pd.isna(x) else x.split(':', 2)[1])

def alignment_search(example, genome):
    if genome == 'tuberculosis_data1':
        for name in orthologs["Tuberculosis_locus"].values:
            if example == name:
                result = orthologs.apply(lambda row: row[orthologs['Tuberculosis_locus'] == name]).values.flatten().tolist()
                return(result)  
    elif genome == 'smegmatis_data1':
        for name in orthologs["smegmatis_locus"].values:
            if example == name:
                result = orthologs.apply(lambda row: row[orthologs['smegmatis_locus'] == name]).values.flatten().tolist()
                return(result)  
    elif genome == 'H37RA_data1':
        for name in orthologs["H37RA_locus"].values:
            if example == name:
                result = orthologs.apply(lambda row: row[orthologs['H37RA_locus'] == name]).values.flatten().tolist()
                return(result)  
    elif genome == 'bovis_data1':
        for name in orthologs["Bovis_locus"].values:
            if example == name:
                result = orthologs.apply(lambda row: row[orthologs['Bovis_locus'] == name]).values.flatten().tolist()
                return(result)  
            
def genome_location(before, after, alignment_search):
    dictionary_values = {}
    for result in alignment_search:
        for locus in tuberculosis_data1["Locus name"].values:
            if result == locus:
                start = int(tuberculosis_data1.loc[tuberculosis_data1["Locus name"] == locus]["Start position"].values) - int(before)
                if start < 0:
                    start = 0
                end = int(tuberculosis_data1.loc[tuberculosis_data1["Locus name"] == locus]["End position"].values) + int(after)
                dictionary_values = {"tuberculosis": [int(start), int(end)]}

        for locus in smagmatis_data1["Locus name"].values:
            if result == locus:
                start = int(smagmatis_data1.loc[smagmatis_data1["Locus name"] == locus]["Start position"].values) - int(before)
                if start < 0:
                    start = 0
                end = int(smagmatis_data1.loc[smagmatis_data1["Locus name"] == locus]["End position"].values) + int(after)
                dictionary_values.update({"smagmatis": [int(start), int(end)]})

        for locus in H37RA_data1["Locus name"].values:
            if result == locus:
                start = int(H37RA_data1.loc[H37RA_data1["Locus name"] == locus]["Start position"].values) - int(before)
                if start < 0:
                    start = 0

                end = int(H37RA_data1.loc[H37RA_data1["Locus name"] == locus]["End position"].values) + int(after)
                dictionary_values.update({"H37RA": [int(start), int(end)]})

        for locus in bovis_data1["Locus name"].values:
            if result == locus:
                start = int(bovis_data1.loc[bovis_data1["Locus name"] == locus]["Start position"].values) - int(before)
                if start < 0:
                    start = 0
                end = int(bovis_data1.loc[bovis_data1["Locus name"] == locus]["End position"].values) + int(after)
                dictionary_values.update({"bovis": [int(start), int(end)]})
    return(dictionary_values)

def custom_fasta(locations_dict, names):
    #We import the modules we need
    from Bio import SeqIO
    from Bio.Align.Applications import MuscleCommandline
    #This one will create names ramdonly 
    import uuid
    
    #We create a fasta file that will contain the sequences to be aligned
    filename = str(uuid.uuid4())
    ofile = open("/Users/dissertation/Documents/Dissertation-real/cgi-mycoapollo/db/genomes/" + filename + ".fasta", "w")

    x = 0
    y = 0
    fasta_result = []

    #In each interaction, we will open the fasta file of the genome and extract the exact parts to be aligned saved
    # in the locations dictionary that we created in the previous function
    for key, value in locations_dict.items():
        if key == 'tuberculosis':
            link_tuberculosis = '/Users/dissertation/Documents/Dissertation-real/cgi-mycoapollo/db/genomes/Tuberculosis.fasta'
            tuberculosis = {rec.id : rec.seq for rec in SeqIO.parse(link_tuberculosis, "fasta")}

            for k, v in tuberculosis.items(): 
                as_string = str(v)
                result = as_string[value[0]:value[1]]
                fasta_result.append(result)

        if key == 'smagmatis':
            link_smagmatis = '/Users/dissertation/Documents/Dissertation-real/cgi-mycoapollo/db/genomes/Smegmatis.fasta'
            smagmatis = {rec.id : rec.seq for rec in SeqIO.parse(link_smagmatis, "fasta")}

            for k, v in smagmatis.items(): 
                as_string = str(v)
                result = as_string[value[0]:value[1]]
                fasta_result.append(result)

        if key == 'H37RA':
            link_H37RA = '/Users/dissertation/Documents/Dissertation-real/cgi-mycoapollo/db/genomes/H37RA.fasta'
            H37RA = {rec.id : rec.seq for rec in SeqIO.parse(link_H37RA, "fasta")}

            for k, v in H37RA.items(): 
                as_string = str(v)
                result = as_string[value[0]:value[1]]
                fasta_result.append(result)

        if key == 'bovis':
            link_bovis = '/Users/dissertation/Documents/Dissertation-real/cgi-mycoapollo/db/genomes/Bovis.fasta'
            bovis = {rec.id : rec.seq for rec in SeqIO.parse(link_bovis, "fasta")}

            for k, v in bovis.items(): 
                as_string = str(v)
                result = as_string[value[0]:value[1]]
                fasta_result.append(result)

    for seq in fasta_result:
        ofile.write(">" + names[x] + "\n" + fasta_result[y] + "\n")
        x +=1
        y +=1

    ofile.close()
    
    muscle_exe = "/Users/dissertation/Documents/Dissertation-real/cgi-mycoapollo/db/muscle3.8.31_i86darwin64"
    in_file = "/Users/dissertation/Documents/Dissertation-real/cgi-mycoapollo/db/genomes/" + filename + ".fasta"
    out_file = "/Users/dissertation/Documents/Dissertation-real/cgi-mycoapollo/db/genomes/" + filename + "_out.fasta"

    muscle_cline = MuscleCommandline(muscle_exe, input=in_file, out=out_file)
    muscle_cline()
    
    return(filename)

def get_colors(seqs):
    """make colors for bases in sequence"""
    text = [i for s in list(seqs) for i in s]
    clrs =  {'A':'red','T':'green','G':'orange','C':'blue','-':'white', 'X':'gray'}
    colors = [clrs[i] for i in text]
    return colors

def view_alignment(aln, fontsize="9pt", plot_width=1500):
    """Bokeh sequence alignment view"""

    #make sequence and id lists from the aln object
    seqs = [rec.seq for rec in (aln)]
    ids = [rec.id for rec in aln]    
    text = [i for s in list(seqs) for i in s]
    colors = get_colors(seqs)    
    N = len(seqs[0])
    S = len(seqs)   
    width = .4

    x = np.arange(1,N+1)
    y = np.arange(0,S,1)
    #creates a 2D grid of coords from the 1D arrays
    xx, yy = np.meshgrid(x, y)
    #flattens the arrays
    gx = xx.ravel()
    gy = yy.flatten()
    #use recty for rect coords with an offset
    recty = gy+.5
    h= 1/S
    #now we can create the ColumnDataSource with all the arrays
    source = ColumnDataSource(dict(x=gx, y=gy, recty=recty, text=text, colors=colors))
    plot_height = len(seqs)*15+100
    x_range = Range1d(0,N+1, bounds='auto')
    if N>100:
        viewlen=100
    else:
        viewlen=N
    #view_range is for the close up view
    view_range = (0,viewlen)
    tools="xpan, reset, save"

    #sequence text view with ability to scroll along x axis
    p1 = figure(title='Alignment result', plot_width=plot_width, plot_height=plot_height,
                x_range=view_range, y_range=ids, tools=tools,
                min_border=0, toolbar_location='below')#, lod_factor=1)          
    glyph = Text(x="x", y="y", text="text", text_align='center',text_color="black",
                text_font_size=fontsize)
    rects = Rect(x="x", y="recty",  width=1, height=1, fill_color="colors",
                line_color=None, fill_alpha=0.4)
    p1.add_glyph(source, glyph)
    p1.add_glyph(source, rects)

    p1.grid.visible = False
    p1.xaxis.major_label_text_font_style = "bold"
    p1.yaxis.minor_tick_line_width = 0
    p1.yaxis.major_tick_line_width = 0

    p = gridplot([[p1]])
    return p

def orf_finder(fasta_file, protein_len):
    record = SeqIO.parse(fasta_file, 'fasta')
    table = 11

    for re in record:
        print(f"\033[1mInformation taken from: " + re.id + "\033[0m")
        for strand, nuc in [(+1, re.seq), (-1, re.seq.reverse_complement())]:
            for frame in range(3):
                length = 3 * ((len(re)-frame) // 3) #Multiple of three
                for pro in nuc[frame:frame+length].translate(table).split("*"):
                    if len(pro) >= min_pro_len:
                        print("%s - length %i, strand %i, frame %i" \
                              % (pro, len(pro), strand, frame) + "\n")
                        
def start_end_codons(sequence):
    seq_dict = {rec.id : rec.seq for rec in SeqIO.parse(sequence, "fasta")}

    start_list = ["ATG"]
    end_list = ["TAA", "TAG", "TGA"]

    # store the starting positions of the codons
    start_positions = []
    end_positions = []

    # note that we can use len() and [] with strings as well, no need to
    # convert to list first
    for k, v in seq_dict.items():
        n = len(v)
        x = 0
        while x < n-2:
            # extract a three-nucleotide subsequence
            possible_codon = v[x:x+3]
            if possible_codon in start_list:
                start_positions.append(x)
            
            elif possible_codon in end_list:
                end_positions.append(x)
            x += 1

        print("Locus:",k)
        print('Found start codons at indices {}'.format(start_positions))
        print('Found end codons at indices {}'.format(end_positions))
        print('Extracted sequence:')
        ext_seq = sequence[start_positions[0]:end_positions[-1]+3]
        ext_seq = str(v)
        ext_seq = ext_seq.replace('ATG', f'\033[1m\033[92mATG\033[0m')
        ext_seq = ext_seq.replace('TAA', f'\033[1m\033[91mTAA\033[0m')
        ext_seq = ext_seq.replace('TAG', f'\033[1m\033[91mTAG\033[0m')
        ext_seq = ext_seq.replace('TGA', f'\033[1m\033[91mTGA\033[0m')
        print(ext_seq)
        print("\n")

def inf_extractor(alignment, bovis_info, tuberculosis_info, h37ra_info, smegmatis_info):
    for locus_alignment in alignment:
        for feature in bovis_info.features:
            if feature.type == "CDS":
                if feature.qualifiers['locus_tag'][0] == locus_alignment:
                    website = """https://www.uniprot.org/uniprot/?query=""" + feature.qualifiers["protein_id"][0] + """&sort=score"""
                    sequence = feature.location.extract(bovis_info).seq
                    content = round((sequence.count('G') + sequence.count('C'))/len(sequence)*100,2)
                    print(f"Information from \033[1mMycobacterium tuberculosis variant bovis AF2122/97 chromosome\033[0m")
                    print(f"\033[1mLocus name:\033[0m", locus_alignment)
                    print(f"\033[1mCoding sequence found at position:\033[0m", feature.location)
                    #print(f"\033[1mGen name:\033[0m", feature.qualifiers["gene"][0])
                    print(f"\033[1mGen sequence:\033[0m", sequence)
                    print(f"\033[1mGC Content:\033[0m", content, "%")
                    print(f"\033[1mGene product:\033[0m", feature.qualifiers["product"][0])
                    print(f"\033[1mProtein ID:\033[0m", feature.qualifiers["protein_id"][0])
                    print(f"\033[1mProtein sequence:\033[0m", feature.qualifiers["translation"][0])
                    print(f"\033[1mNote:\033[0m", feature.qualifiers["note"][0])
                    if feature.qualifiers["inference"][0] != '':
                        print(f"\033[1mFurther information:\033[0m", feature.qualifiers["inference"][0])
                    display(Markdown("[Uniprot](" + website + ")"))

        for feature in tuberculosis_info.features:
            if feature.type == "CDS":
                if feature.qualifiers['locus_tag'][0] == locus_alignment:
                    website = """https://www.uniprot.org/uniprot/?query=""" + feature.qualifiers["protein_id"][0] + """&sort=score"""
                    sequence = feature.location.extract(tuberculosis_info).seq
                    content = round((sequence.count('G') + sequence.count('C'))/len(sequence)*100,2)
                    print(f"Information from \033[1mMycobacterium tuberculosis H37Rv complete genome\033[0m")
                    print(f"\033[1mLocus name:\033[0m", locus_alignment)
                    print(f"\033[1mCoding sequence found at position:\033[0m", feature.location)
                    print(f"\033[1mGen name:\033[0m", feature.qualifiers["gene"][0])
                    print(f"\033[1mGen sequence:\033[0m", sequence)
                    print(f"\033[1mGC Content:\033[0m", content, "%")
                    print(f"\033[1mGene product:\033[0m", feature.qualifiers["product"][0])
                    print(f"\033[1mProtein ID:\033[0m", feature.qualifiers["protein_id"][0])
                    print(f"\033[1mProtein sequence:\033[0m", feature.qualifiers["translation"][0])
                    print(f"\033[1mNote:\033[0m", feature.qualifiers["note"][0])
                    display(Markdown("[Uniprot](" + website + ")"))
 
        for feature in h37ra_info.features:
            if feature.type == "CDS":
                if feature.qualifiers['locus_tag'][0] == locus_alignment:
                    website = """https://www.uniprot.org/uniprot/?query=""" + feature.qualifiers["protein_id"][0] + """&sort=score"""
                    sequence = feature.location.extract(h37ra_info).seq
                    content = round((sequence.count('G') + sequence.count('C'))/len(sequence)*100,2)
                    print(f"Information from \033[1mMycobacterium tuberculosis H37Ra, complete sequence\033[0m")
                    print(f"\033[1mLocus name:\033[0m", locus_alignment)
                    print(f"\033[1mCoding sequence found at position:\033[0m", feature.location)
                    #print(f"\033[1mGen name:\033[0m", feature.qualifiers["gene"][0])
                    print(f"\033[1mGen sequence:\033[0m", sequence)
                    print(f"\033[1mGC Content:\033[0m", content, "%")
                    print(f"\033[1mGene product:\033[0m", feature.qualifiers["product"][0])
                    print(f"\033[1mProtein ID:\033[0m", feature.qualifiers["protein_id"][0])
                    print(f"\033[1mProtein sequence:\033[0m", feature.qualifiers["translation"][0])
                    print(f"\033[1mNote:\033[0m", feature.qualifiers["note"][0])
                    if feature.qualifiers["inference"][0] != '':
                        print(f"\033[1mFurther information:\033[0m", feature.qualifiers["inference"][0])
                    display(Markdown("[Uniprot](" + website + ")"))

        for feature in smegmatis_info.features:
            if feature.type == "CDS" or feature.type == "Gen":
                if feature.qualifiers['locus_tag'][0] == locus_alignment:
                    website = """https://www.uniprot.org/uniprot/?query=""" + feature.qualifiers["protein_id"][0] + """&sort=score"""
                    sequence = feature.location.extract(smegmatis_info).seq
                    if sequence != '':
                        content = round((sequence.count('G') + sequence.count('C'))/len(sequence)*100,2)
                    print(f"Information from \033[1mMycolicibacterium smegmatis MC2 155, complete sequence\033[0m")
                    print(f"\033[1mLocus name:\033[0m", locus_alignment)
                    print(f"\033[1mCoding sequence found at position:\033[0m", feature.location)
                    #print(f"\033[1mGen name:\033[0m", feature.qualifiers["gene"][0])
                    print(f"\033[1mGen sequence:\033[0m", sequence)
                    if sequence != '':
                        print(f"\033[1mGC Content:\033[0m", content, "%")
                    print(f"\033[1mGene product:\033[0m", feature.qualifiers["product"][0])
                    print(f"\033[1mProtein ID:\033[0m", feature.qualifiers["protein_id"][0])
                    print(f"\033[1mProtein sequence:\033[0m", feature.qualifiers["translation"][0])
                    print(f"\033[1mNote:\033[0m", feature.qualifiers["note"][0])
                    if feature.qualifiers["inference"][0] != '':
                        print(f"\033[1mFurther information:\033[0m", feature.qualifiers["inference"][0])
                    display(Markdown("[Uniprot](" + website + ")"))

This is where you need to choose the gen name that you want to check and the genome where the gen is located.<br />    - Mycobacterium tuberculosis H37Ra = **H37RA_data1**
<br />    - Mycobacterium tuberculosis H37Rv = **tuberculosis_data1**
<br />    - Mycolicibacterium smegmatis MC2155 = **smegmatis_data1**
<br />    - Mycobacterium tuberculosis variant bovis AF2122/97 = **bovis_data1**

The **before** and **after** indicates if you want to get an alignment before or after the ortholog starts.

If you don't want to star a few bases before the ortholog leave this as 0. Same goes if you wish it to finish afther the ortholog finishes.

In [128]:
example = 'Rv3197A'
genome = 'tuberculosis_data1'
#genome = 'H37RA_data1'
#genome = 'smegmatis_data1'
#genome = 'bovis_data1'

before = '0'
after = '0'

You shouldn't change anything from here, just run it when you have selected the gen that you need information from

In [129]:
alignment = alignment_search(example, genome)

genome = genome_location(before, after, alignment)

filename = custom_fasta(genome, alignment)

info = AlignIO.read('/Users/dissertation/Documents/Dissertation-real/cgi-mycoapollo/db/genomes/' + filename + '_out.fasta','fasta')

from Bio.Align import AlignInfo
summary_align = AlignInfo.SummaryInfo(info)
consensus = summary_align.dumb_consensus()

count_NA = alignment.count('N/A')
size = len(alignment)

result = size - count_NA

new_info = open('/Users/dissertation/Documents/Dissertation-real/cgi-mycoapollo/db/genomes/' + filename + '_out.fasta', "a")
new_info.write(">Consensus" + "\n" + str(consensus))
new_info.close()

if result == 1:
    print('The gen you selected does not have any orthologs to display.')
else:
    aln = AlignIO.read('/Users/dissertation/Documents/Dissertation-real/cgi-mycoapollo/db/genomes/' + filename + '_out.fasta','fasta')
    p = view_alignment(aln, plot_width=900)
    pn.pane.Bokeh(p)
    show(p)

#### Part 2. Annotation about the genes

Run this if you want to get information about the alignment you have obtained.

**Note**: You only need to run this once.

**Further information functionality**: Mycobacterium tuberculosis H37Rv does not have the further information display as some genes/locus do not work properly. This must be due to some issues with the genbank file and not the code.

The sequence for dnaA for Mycolicibacterium smegmatis MC2 155 is not available on the genbank file, so it won't show.

In [108]:
bovis_info = SeqIO.read('/Users/dissertation/Documents/Dissertation-real/cgi-mycoapollo/db/genomes/bovis_tuberculosis.gb', "genbank")
tuberculosis_info = SeqIO.read('/Users/dissertation/Documents/Dissertation-real/cgi-mycoapollo/db/genomes/tuberculosis_sequence.gb', "genbank")
h37ra_info = SeqIO.read('/Users/dissertation/Documents/Dissertation-real/cgi-mycoapollo/db/genomes/H37RA.gbk', "genbank")
smegmatis_info = SeqIO.read('/Users/dissertation/Documents/Dissertation-real/cgi-mycoapollo/db/genomes/sequence.gb', "genbank")

inf_extractor(alignment, bovis_info, tuberculosis_info, h37ra_info, smegmatis_info)

Information from Mycobacterium tuberculosis H37Rv complete genome
Locus name: Rv3197A
Coding sequence found at position: [3568400:3568679](-)
Gen name: whiB7
Gen sequence: GTGTCGGTACTGACAGTCCCCAGACAGACCCCCAGACAAAGATTGCCGGTTTTGCCGTGCCACGTCGGTGATCCCGATCTGTGGTTCGCCGATACCCCGGCCGGTCTCGAGGTAGCCAAGACACTGTGTGTGAGCTGCCCGATCAGGCGGCAGTGCTTGGCCGCGGCGCTTCAGCGGGCTGAACCCTGGGGCGTTTGGGGTGGTGAGATATTCGACCAAGGCTCGATCGTGAGTCACAAGCGTCCGCGCGGACGTCCGCGCAAGGATGCTGTTGCATAG
GC Content: 62.72 %
Gene product: Probable transcriptional regulatory protein WhiB-like WhiB7
Protein ID: CCP46011.1
Protein sequence: MSVLTVPRQTPRQRLPVLPCHVGDPDLWFADTPAGLEVAKTLCVSCPIRRQCLAAALQRAEPWGVWGGEIFDQGSIVSHKRPRGRPRKDAVA
Note: Rv3197A, len: 92 aa. Probable whiB7 (alternate gene name: whmC), WhiB-like regulatory protein (see citation below), similar to WhiB paralogue of Streptomyces coelicolor, wblE gene product (85 aa). Equivalent to Q49765|WHIB7|ML0639|B1937_F2_68 putative transcriptional regulator WHIB7 from Mycobacterium leprae (89 

[Uniprot](https://www.uniprot.org/uniprot/?query=CCP46011.1&sort=score)

Information from Mycolicibacterium smegmatis MC2 155, complete sequence
Locus name: MSMEG_RS09480
Coding sequence found at position: [2031776:2032088](+)
Gen sequence: ATGTCCATTGCGATGACTGCTCCGACCACGGGCGTCGCGCCGATGACATGCGAGACGCGACTGCCGGCGGTGCCGTGCCATGTCGGTGACCCGGATCTGTGGTTCGCCGAGAACCCCGGCGACCTTGAGCGGGCCAAGGCCCTGTGCGCGGGGTGCCCGATCCGTGTGCAGTGCCTGACCGCGGCGCTCGAACGGCAGGAACCGTGGGGTGTCTGGGGTGGCGAGATCCTCGACCGCGGAAGCATTGTCGCGCGCAAGCGTCCGCGCGGGCGTCCGCGTAAGGATTCCGGCGGCAACCCGGCCGCCGCCTGA
GC Content: 70.51 %
Gene product: transcriptional regulator WhiB7
Protein ID: WP_003893333.1
Protein sequence: MSIAMTAPTTGVAPMTCETRLPAVPCHVGDPDLWFAENPGDLERAKALCAGCPIRVQCLTAALERQEPWGVWGGEILDRGSIVARKRPRGRPRKDSGGNPAAA
Note: Derived by automated computational analysis using gene prediction method: Protein Homology.
Further information: COORDINATES: similar to AA sequence:RefSeq:WP_003893333.1


[Uniprot](https://www.uniprot.org/uniprot/?query=WP_003893333.1&sort=score)

Information from Mycobacterium tuberculosis H37Ra, complete sequence
Locus name: MRA_RS17005
Coding sequence found at position: [3579183:3579462](-)
Gen sequence: GTGTCGGTACTGACAGTCCCCAGACAGACCCCCAGACAAAGATTGCCGGTTTTGCCGTGCCACGTCGGTGATCCCGATCTGTGGTTCGCCGATACCCCGGCCGGTCTCGAGGTAGCCAAGACACTGTGTGTGAGCTGCCCGATCAGGCGGCAGTGCTTGGCCGCGGCGCTTCAGCGGGCTGAACCCTGGGGCGTTTGGGGTGGTGAGATATTCGACCAAGGCTCGATCGTGAGTCACAAGCGTCCGCGCGGACGTCCGCGCAAGGATGCTGTTGCATAG
GC Content: 62.72 %
Gene product: transcriptional regulator WhiB7
Protein ID: WP_003899963.1
Protein sequence: MSVLTVPRQTPRQRLPVLPCHVGDPDLWFADTPAGLEVAKTLCVSCPIRRQCLAAALQRAEPWGVWGGEIFDQGSIVSHKRPRGRPRKDAVA
Note: Derived by automated computational analysis using gene prediction method: Protein Homology.
Further information: COORDINATES: similar to AA sequence:RefSeq:YP_177940.1


[Uniprot](https://www.uniprot.org/uniprot/?query=WP_003899963.1&sort=score)

Information from Mycobacterium tuberculosis variant bovis AF2122/97 chromosome
Locus name: BQ2027_RS16470
Coding sequence found at position: [3527356:3527635](-)
Gen sequence: GTGTCGGTACTGACAGTCCCCAGACAGACCCCCAGACAAAGATTGCCGGTTTTGCCGTGCCACGTCGGTGATCCCGATCTGTGGTTCGCCGATACCCCGGCCGGTCTCGAGGTAGCCAAGACACTGTGTGTGAGCTGCCCGATCAGGCGGCAGTGCTTGGCCGCGGCGCTTCAGCGGGCTGAACCCTGGGGCGTTTGGGGTGGTGAGATATTCGACCAAGGCTCGATCGTGAGTCACAAGCGTCCGCGCGGACGTCCGCGCAAGGATGCTGTTGCATAG
GC Content: 62.72 %
Gene product: transcriptional regulator WhiB7
Protein ID: WP_003899963.1
Protein sequence: MSVLTVPRQTPRQRLPVLPCHVGDPDLWFADTPAGLEVAKTLCVSCPIRRQCLAAALQRAEPWGVWGGEIFDQGSIVSHKRPRGRPRKDAVA
Note: Derived by automated computational analysis using gene prediction method: Protein Homology.
Further information: COORDINATES: similar to AA sequence:RefSeq:YP_177940.1


[Uniprot](https://www.uniprot.org/uniprot/?query=WP_003899963.1&sort=score)

#### Part 3. Further information regarding sequences

The following code will highlight all the instances found for the start codon ATG and it will print the positions and in green bold color.

As usual, you only need to run this once per search.

In [67]:
aln = '/Users/dissertation/Documents/Dissertation-real/cgi-mycoapollo/db/genomes/' + filename + '.fasta'

start_end_codons(aln)

Locus: rrl
Found start codons at indices [146, 338, 381, 423, 539, 898, 977, 1192, 1315, 1375, 1396, 1512, 1564, 1587, 1746, 1804, 2165, 2183, 2315, 2435, 2841, 2925, 3105]
Found end codons at indices [5, 21, 47, 103, 110, 143, 246, 270, 286, 313, 336, 475, 512, 586, 602, 634, 654, 679, 703, 761, 821, 824, 858, 877, 903, 912, 973, 984, 1011, 1041, 1054, 1124, 1127, 1176, 1242, 1248, 1266, 1280, 1325, 1346, 1418, 1451, 1489, 1569, 1578, 1656, 1734, 1863, 1896, 1903, 1939, 1986, 1997, 2006, 2104, 2113, 2142, 2180, 2217, 2266, 2320, 2423, 2442, 2501, 2511, 2579, 2749, 2784, 2798, 2853, 2921, 2976, 3036, 3114]
Extracted sequence:
TGTTGTAAGTTTTCGGCCGGTTAGTGCCAGTTCCCTGCACCTATTACTAGGCTTCCAGGTCTGGTCTATCAATCCCGTGGTCTGCGGGGGGCCTTATCCCTCCTAGAGGGTGAGAAGCCTGGTCTTGGAAGAGGTTTCCCGCTTAGATGCTTTCAGCGGTTATCCTGTCCGAACGTGGCTATCCAGCCGTGCCCCTGGTGGGACAACTGGTATACCAGAGGTTCGTCCGTCCCGGTCCTCTCGTACTAGGGACAGGTTTCCTCAAGCTTCTGACGCGCGCGGCGGATAGAGACCGAACTGTCTCACGACGTTCTAAACCCAGCTCGCGTGCCGCTTTAATGGGCGAACAGCCCAACCCTTGGGACC

Although the previous code shows the start and end codon for each sequence. The following code will show all available ORF on the sequences we have obtained.

If you wish to modify the protein lenght just change the number on the third line to whichever number you want to obtain.

In [68]:
record = '/Users/dissertation/Documents/Dissertation-real/cgi-mycoapollo/db/genomes/' + filename + '.fasta'
min_pro_len = 100

orf_finder(record, min_pro_len)

Information taken from: rrl
WANSPTLGTCSSPRMRRADIEVPNHPVDMDSWGRSACYPRGTFYPLSDTPSTQRCRITSPDFRPCSTCPSRSQAPLCTYTQHLIAVQVEGTFGRLRYILGGNRPS - length 105, strand 1, frame 0

YSLPDHLCWFGVRAVYVLARGFSRQHRITEFASLGYASPLRIYARRIYLSDSLQAYPGITTDRYGYLPASPHRLTTTNEGPMQPHQRHTPKGAVTVVLDG - length 100, strand 1, frame 0

PSSTGQASVRIHRLATSHGPVFLVNSRFSLVCATHSRLGRKGLHAMWVPLLPKLRGHFAEFLNHSSLVRLSILYLTTCVGLGYGPCMSSLEAFLGSIGSPNSPHSAMHHLSGYMPDGFTYLTPYRLTPVLPLTGTATFLRHPIA - length 144, strand 1, frame 1

LLPTKVPCSPTNATPRKVRSRWFWMVSTADSSWTHIHGYGNINPLSIDYACRPRLRSRLTLGGLAWPRNPWSFGGQGSHLPNRYSCLHSHSHTLHHSITRRLHRMQDAPLPSQHVGCRGFGGVLEPRYIIGAQSLDQ - length 137, strand 1, frame 1

ITPLRVQNTPLHHTLLCGYALFRLAFAAATPPGLTSRHVPDSQAHSSKGTPSPHDTSRALTDCKRTVSGTLSLPSRGTFHHSLTVLIRYRSLGSIQAYRVVPADSQQIPRARCYSGTPSRQTSGFHVPGSHRLRQAIPSHFR - length 142, strand 1, frame 1

PQHFLTALQPGRAGKIGPTTPHTQPLPGITCIRFSHPPLSLATTHGITFCFLFLRVLRCFTSPRSPQTPIYSAPGDTTSLVPGFPIRTSSDQRSVDSSPRLIAASYVLHRLPVPRHPPCALKHLQ - length 125, strand 1, frame 1

MLSAVILSERGYPAV